# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [22]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [55]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [56]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [57]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())
ed.links

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [58]:
link_system_prompt = """You are provided with a list of links found on a webpage.
Your task is to determine which links would be most relevant to include in a brochure about the company,
such as links to an About page, Company page, or Careers/Jobs page.

You should respond in JSON format like this:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}

Below are a few examples:

Example 1:
Input:
[
    "https://example.com/",
    "https://example.com/about",
    "https://example.com/products",
    "https://example.com/careers",
    "https://example.com/blog"
]
Output:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
Input:
[
    "https://company.org/",
    "https://company.org/team",
    "https://company.org/jobs",
    "https://company.org/contact",
    "https://company.org/privacy-policy"
]
Output:
{
    "links": [
        {"type": "about page", "url": "https://company.org/team"},
        {"type": "careers page", "url": "https://company.org/jobs"}
    ]
}

Example 3:
Input:
[
    "https://startup.co/",
    "https://startup.co/who-we-are",
    "https://startup.co/services",
    "https://startup.co/careers",
    "https://startup.co/legal"
]
Output:
{
    "links": [
        {"type": "about page", "url": "https://startup.co/who-we-are"},
        {"type": "careers page", "url": "https://startup.co/careers"}
    ]
}

Now, analyze the next list of links and respond in the same JSON format.
"""

In [59]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.
Your task is to determine which links would be most relevant to include in a brochure about the company,
such as links to an About page, Company page, or Careers/Jobs page.

You should respond in JSON format like this:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}

Below are a few examples:

Example 1:
Input:
[
    "https://example.com/",
    "https://example.com/about",
    "https://example.com/products",
    "https://example.com/careers",
    "https://example.com/blog"
]
Output:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
Input:
[
    "https://company.org/",
    "https://company.org/team",
    "https://company.org/jobs",
    "https://company.org/contact",
    "https://c

In [60]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [61]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [62]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [63]:
get_links("https://www.anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}

In [64]:
anthropic = Website("https://www.anthropic.com")
for link in anthropic.links:
    print(link)

#main-content
#footer
/
/news
https://claude.ai/
https://www.anthropic.com/research#entry:8@1:url
https://www.anthropic.com/claude
https://claude.ai/
/api
https://www.anthropic.com/news/claude-3-7-sonnet
/claude/sonnet
/news/visible-extended-thinking
/enterprise
/news/constitutional-ai-harmlessness-from-ai-feedback
/news/core-views-on-ai-safety
/jobs
/
/claude
/team
/enterprise
/education
https://claude.ai/download
/pricing
http://claude.ai/login
/api
https://docs.anthropic.com/
/amazon-bedrock
/google-cloud-vertex-ai
/pricing#anthropic-api
https://console.anthropic.com/
/research
/economic-index
/claude/sonnet
/claude/haiku
/news/claude-3-family
/transparency
/responsible-scaling-policy
https://trust.anthropic.com
/solutions/agents
/solutions/coding
/solutions/customer-support
/news
/customers
/engineering
/learn
/company
/careers
https://status.anthropic.com/
/supported-countries
https://support.anthropic.com
/legal/privacy
/responsible-disclosure-policy
/legal/consumer-terms
/legal/

In [65]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/all-hands/openhands-lm-32b-v0.1',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/ByteDance/MegaTTS3',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/virtuoussy/Multi-subject-RLVR',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/m

In [66]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [67]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [68]:
print(get_all_details("https://www.anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
API
Solutions
Research
Commitments
Learn
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Claude 3.7 Sonnet and Claude Code
Introducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.
Read the announcement
Model details
Claude 3.7 Sonnet
Research insights
Claude's extended thinking
Our work
Product
Claude for Enterprise
Sep 04, 2024
Research
Constitutional AI: Harmlessness from AI Fee

In [74]:
system_prompt = """You are a helpful assistant creating a fun and easy-to-read brochure for kids about a company.
Use the information provided from the company's website to explain what the company does, why it’s exciting, and why kids might like it.

Make the brochure in markdown format with fun sections like:

# What This Company Does  
(A kid-friendly explanation of the company’s product or service)

# Why It's Awesome  
(Explain what's cool, fun, or inspiring about the company)

# Who Works Here  
(Talk about the people and the company culture in a friendly way)

# Can I Work Here Too?  
(A section about jobs or careers explained in a way that kids understand—like, "When you grow up, you could work here if you like [X]!")

# Who Uses This Company  
(Explain who the company helps or who its customers are)

Be clear, positive, and make it interesting for a 10-year-old to read. Include emojis if it helps make it more fun 🎉😄🚀

**Translate the entire brochure into Spanish.**"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [75]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are learning about a company called **{company_name}**!\n"
    user_prompt += "Here is everything we found on their website. Use this information to help you write a fun brochure about the company for kids.\n\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

In [76]:
get_brochure_user_prompt("Anthropic", "https://www.anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}


"You are learning about a company called **Anthropic**!\nHere is everything we found on their website. Use this information to help you write a fun brochure about the company for kids.\n\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nAPI\nSolutions\nResearch\nCommitments\nLearn\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.7 Sonnet\nClaude 3.7 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nClaude 3.7 Sonnet and Claude Code\nIntroducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.\nRead the announcement\nModel details\nClaude 3.7 Sonnet\nResearch insights\nClaude's extended thinking\nOur work\nProduct\nClaude for Enterprise\nSe

In [77]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [78]:
create_brochure("Anthropic", "https://www.anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}


```markdown
# 🎉 Welcome to Anthropic! 🚀

## What This Company Does
At Anthropic, we are all about **AI** - that stands for Artificial Intelligence! 🤖 We create super-smart computer programs that can understand and help you with all sorts of cool tasks! Our newest invention is called **Claude 3.7 Sonnet**, and it's like having a wise robot friend who can chat with you, help you code, and solve problems! 🌟 We believe that AI can be really helpful if it’s made safely and smartly.

## Why It's Awesome
Imagine a world where you can have a smart robot buddy that can help you with your homework or even tell you stories! 📚✨ At Anthropic, we make sure our robots like Claude are reliable and safe to use. We like asking big questions about how AI can make the world better and safer! Plus, we're like detectives, always exploring new ways to make AI even cooler and more trustworthy! 🔍💡

## Who Works Here
Our team is made up of super smart people from all kinds of backgrounds! 🧑‍🔬👩‍💻 We've got researchers, engineers, and even experts in rules and policies, all working together to build amazing AI. Everyone here loves learning and sharing their ideas, and guess what? We also love plants! 🌿 It's a friendly place where everyone pitches in to make the world a better place through AI.

## Can I Work Here Too?
Absolutely! When you grow up, if you like things like coding, problem-solving, or even helping others, you could join Anthropic! 🌈 We always look for passionate people who want to make smart AI systems even better! So, if you dream of working with robots, this could be the place for you! 🚀

## Who Uses This Company
Our awesome AI systems help different groups around the world! 🌍 From businesses to schools, our technology helps people do their work better and faster. We want to make sure that everyone can enjoy the benefits of safe and friendly AI. So whether it’s helping a student with their homework or assisting a business with their tasks, Claude has got it covered! 🏫🏢

Thank you for joining us today! Remember, with the help of Anthropic and our amazing AI, the future is bright and full of possibilities! 🌟
```

---

```markdown
# 🎉 ¡Bienvenido a Anthropic! 🚀

## ¿Qué Hace Esta Empresa?
En Anthropic, nos dedicamos a la **IA** - ¡eso significa Inteligencia Artificial! 🤖 Creamos programas de computadora súper inteligentes que pueden entenderte y ayudarte con todo tipo de tareas geniales. ¡Nuestra invención más nueva se llama **Claude 3.7 Sonnet**, y es como tener un amigo robot sabio que puede charlar contigo, ayudarte a programar y resolver problemas! 🌟 Creemos que la IA puede ser realmente útil si se crea de forma segura y inteligente.

## ¿Por Qué Es Genial?
¡Imagina un mundo donde puedes tener un inteligente amigo robot que te ayude con tu tarea o incluso te cuente historias! 📚✨ En Anthropic, aseguramos que nuestros robots como Claude sean confiables y seguros de usar. Nos gusta hacernos grandes preguntas sobre cómo la IA puede hacer del mundo un lugar mejor y más seguro. ¡Además, somos como detectives, siempre explorando nuevas maneras de hacer que la IA sea aún más genial y confiable! 🔍💡

## ¿Quién Trabaja Aquí?
Nuestro equipo está formado por personas súper inteligentes de todo tipo de campos. 🧑‍🔬👩‍💻 ¡Tenemos investigadores, ingenieros e incluso expertos en reglas y políticas, todos trabajando juntos para construir una IA increíble! Todos aquí aman aprender y compartir sus ideas, ¡y adivina qué! ¡También amamos las plantas! 🌿 Es un lugar amigable donde todos colaboran para hacer del mundo un lugar mejor a través de la IA.

## ¿Puedo Trabajar Aquí También?
¡Absolutamente! Cuando crezcas, si te gustan cosas como programar, resolver problemas o incluso ayudar a otros, podrías unirte a Anthropic. 🌈 ¡Siempre buscamos personas apasionadas que quieran hacer que los sistemas de IA sean aún mejores! Así que, si sueñas con trabajar con robots, ¡este podría ser el lugar para ti! 🚀

## ¿Quién Usa Esta Empresa?
Nuestros increíbles sistemas de IA ayudan a diferentes grupos en todo el mundo. 🌍 Desde negocios hasta escuelas, nuestra tecnología ayuda a las personas a hacer su trabajo mejor y más rápido. ¡Queremos asegurarnos de que todos puedan disfrutar de los beneficios de una IA segura y amigable! Así que ya sea ayudando a un estudiante con su tarea o asistiendo a un negocio con sus tareas, ¡Claude se encarga de todo! 🏫🏢

¡Gracias por acompañarnos hoy! Recuerda, con la ayuda de Anthropic y nuestra increíble IA, ¡el futuro es brillante y lleno de posibilidades! 🌟
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("Anthropic", "https://www.anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Company Brochure

## About Us
Welcome to Anthropic! We are an innovative AI safety and research company based in San Francisco, dedicated to building reliable, interpretable, and steerable AI systems. Our interdisciplinary team of researchers, engineers, policy experts, and operational leaders collaborate to generate impactful research and create AI products that prioritize safety and reliability.

## Our Purpose
At Anthropic, we believe that AI will have a profound influence on the world. We strive to create systems that people can trust and are committed to understanding the opportunities and risks associated with AI technologies. We treat AI safety as a systematic science, continuously applying our research insights to our products and sharing our findings with the global community.

## Our Products
- **Claude**: We are excited to introduce **Claude 3.7 Sonnet**, our most advanced AI model, designed to help you build AI-powered applications and custom experiences. Claude also features a special coding tool called **Claude Code**, providing developers with a powerful agentic resource for programming tasks.
  
- **API Solutions**: Our API allows businesses to integrate Claude into their applications, enhancing capabilities for customer support, coding, and more.

## Interdisciplinary Collaboration
We view ourselves as one integral part of the broader AI ecosystem and work closely with civil society, government, academia, and industry to advance industry-wide safety measures. Our diverse team brings a wealth of experience from various sectors, such as NASA and startups, fostering an environment of innovation and creativity.

## Our Commitment to Safety
Safety is at the core of everything we do. We invest in frontier research and innovative safety techniques to ensure our AI systems are reliable and beneficial. Our commitment encompasses not only our products but also our interactions with the world, as we actively communicate our findings to policymakers and stakeholders.

## Company Culture
At Anthropic, we believe in a collaborative and inclusive environment where diverse perspectives are valued. Our team members have a shared passion for AI and its potential impact. We prioritize transparency, ethical practices, and continuous growth. Plus, we have a fondness for plants!

## Careers at Anthropic
We're always on the lookout for talented and motivated individuals to join our mission. Whether your expertise lies in research, engineering, policy, or operations, we offer a stimulating and rewarding workplace. Explore our open roles and consider being part of our journey to shape the future of AI responsibly.

### Join Us Today!
Visit our website to learn more about our products, research, and career opportunities. Together, we can build the future of AI—safely and effectively.

---

**Follow Us:**
- [Website](https://www.anthropic.com)
- [LinkedIn](https://www.linkedin.com/company/anthropic) 
- [Twitter](https://twitter.com/anthropic) 

**Contact Us:**
Email: info@anthropic.com   
Location: San Francisco, CA  

---

Thank you for considering Anthropic as a partner, customer, or potential workplace. We look forward to connecting with you!

In [35]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# Welcome to Hugging Face! 😊

## **The AI Community Building the Future 🚀**

At Hugging Face, we like our AI models like we like our hugs: warm, friendly, and helping build a brighter future! From datasets to models, we bring the AI community together for the ultimate collaboration experience.

---

### **What We Do:**
- **Models Galore!** 🤖
  - With over **1 million** models at your disposal, you're sure to find one that matches your unique quirks! Trending this week: Don't miss our celebrity, *Qwen/Qwen2.5-Omni-7B*! 🌟

- **Daring Datasets 📊**
  - Browse through an impressive collection of **250k+ datasets**! Each more exciting than watching paint dry when it finally dries. 🚥

- **Spaces for Days** 🏗️
  - Create, discover, and collaborate in our vibrant Spaces. It's like the *Facebook* of AI, but without the scandals! 

---

### **Join Our Happy Family** 👨‍👩‍👧‍👦
- **Come for the work, stay for the community!** At Hugging Face, we believe that every voice contributes to something special.
- **Fun Fact:** We provide enterprise solutions that give businesses the AI tools they need while ensuring our employees have more focus than caffeinated cats on a laser pointer!

---

### **Looking for a Career?**
- Join a team of over **211 Hugging Faces** who are working hard and hardly working on creating the future of machine learning. 🥳
- Positions range from model wranglers to dataset pioneers. So if you're looking for a career path that isn't "hugs only," we might just have it!

---

### **What Our Customers Say:**
- Over **50,000 organizations** choose Hugging Face as their AI partner. That’s almost as impressive as getting featured on the latest TikTok trend! 🌐
- From *AI at Meta* to *Google*: If you’ve got brainy ideas, we’ve got a hug for you.

---

### **We’re A Little Quirky** 😜
- Our open-source approach means that we collaborate and share the love. We even have models like *Transformers* and *Diffusers*. Who says AI can't have fun with names?!

---

### **Join Us Today!** 🌈
**Sign Up to start your AI adventure!** We promise we won’t give you unsolicited hugs—unless you like that sort of thing. 

---

🏆 Hugging Face: Hugs not bugs! 🏆
Come get your "hug" of the latest AI trends and tools—where the future is just a little bit warmer!



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [17]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [18]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [19]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [20]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
import gradio as gr
import requests
from openai import OpenAI
import anthropic

openai = OpenAI()
claude = anthropic.Anthropic()

view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model"),
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.
